In [1]:
import os
import requests
import s3fs

#Set up configuration variables
caltechdata_url = 'https://data.caltechlibrary.dev/api/records'
prefix = '10.33569/'
osn_endpoint = 'https://renc.osn.xsede.org/'
osn_path = 'ini210004tommorrell/tomography_archive/'
local_directory = 'data'


In [2]:
#Query CaltechDATA to get identifiers of items of interest

keyword = "Bdellovibrio bacteriovorus"

data = {"q":f"metadata.subjects.subject={keyword}"}

response = requests.get(caltechdata_url,data)

hits = response.json()['hits']
print(f"{hits['total']} record found")


tiltids = {}
for hit in hits['hits']:
    doi = hit['pids']['doi']['identifier']
    tiltid = hit['metadata']['identifiers'][0]['identifier']
    tiltids[tiltid] = doi
    

1 record found


In [ ]:
# Download files directly from Open Storage Network

#Set up local directory for files
if not os.path.isdir(local_directory):
    os.mkdir(local_directory)
os.chdir(local_directory)

# Copy files using S3 interface
osn_s3 = s3fs.S3FileSystem(anon=True, client_kwargs={"endpoint_url": osn_endpoint})

for tiltid in tiltids:
    print(f'Downloading {tiltid}')
    osn_s3.get(osn_path+tiltid, '.',recursive=True)
